# Depth Validation

validating the use of depth information in rgb-d and rgb-hha models

In [1]:
import time
import itertools

from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
import tensorflow_hub as hub

from src import utils#, eval_utils, rgbd_model, rgb_model, depth_model, hha_model, rgb_hha_model

## Load dataset(s)

In [2]:
train_data_path = 'C:/Users/awnya/Documents/Projects/RGBD Object Classification/RGBD_dataset' + '/train'
test_data_path = 'C:/Users/awnya/Documents/Projects/RGBD Object Classification/RGBD_dataset' + '/test'

In [3]:
#
start_time = time.time()

print('loading rgbd test dataset')
# rgbd_train_dataset = utils.create_rgbd_dataset(train_data_path, azure=False)
rgbd_test_dataset = utils.create_rgbd_dataset(test_data_path, azure=False)

print(f'done! took {time.time()-start_time:.2f} seconds')

loading rgbd test dataset
done! took 23.84 seconds


In [4]:
#
start_time = time.time()

print('loading rgb-hha test dataset')
# rgbd_train_dataset = utils.create_rgbd_dataset(train_data_path, azure=False)
rgb_hha_test_dataset = utils.create_rgb_hha_dataset(test_data_path, azure=False)

print(f'done! took {time.time()-start_time:.2f} seconds')

loading rgb-hha test dataset
done! took 16.80 seconds


## Load model(s)

In [5]:
import sys
sys.path.insert(0,'src')

In [6]:
import rgb_hha_model, rgbd_model

In [7]:
rgb_feat_vec_embedding = hub.KerasLayer('models/bit_m-r50x1_1')

In [8]:
hha_weights_path = 'src/models_archive/hha_run2_epoch6_weights.h5'

hha_feat_extractor = rgb_hha_model.build_hha_feature_extractor(
    weights=hha_weights_path, trainable=True)
# hha_feat_extractor.summary()

# now build rgb-hha model
rgb_hha_mdl = rgb_hha_model.build_rgb_hha_model(input_shape=(200, 200), num_classes=51,
    hha_feat_vec_embedding=hha_feat_extractor, rgb_feat_vec_embedding=rgb_feat_vec_embedding,
    model_name='rgb-hha_model')

In [9]:
rgbd_mdl = rgbd_model.build_model(input_shape=(200, 200), num_classes=51, 
    feat_vec_embedding=rgb_feat_vec_embedding)

In [10]:
rgbd_weights = 'src/models_archive/rgbd_model_run29_epoch3_weights.h5'
rgbd_mdl.load_weights(rgbd_weights)

## Examine Trained weights

### rgb-hha

In [11]:
dense1_weights = rgb_hha_mdl.get_layer(name='dense1').weights[0]
dense1_weights

<tf.Variable 'dense1/kernel:0' shape=(2624, 128) dtype=float32, numpy=
array([[ 0.02306938,  0.03978315, -0.00144333, ..., -0.00076058,
         0.02449546, -0.00484867],
       [-0.03843796, -0.00768983, -0.01515529, ..., -0.00361391,
         0.04495641,  0.03982913],
       [-0.03658483,  0.00489531, -0.03869307, ..., -0.04374599,
        -0.0307797 ,  0.01834197],
       ...,
       [ 0.03150812,  0.0091397 , -0.03175697, ...,  0.04335028,
         0.03267912,  0.0101967 ],
       [ 0.01170947,  0.03749277,  0.01848451, ..., -0.00074663,
        -0.03179657, -0.03446211],
       [ 0.02998374, -0.01455357, -0.04189331, ..., -0.03203732,
        -0.01371172, -0.01200302]], dtype=float32)>

In [12]:
rgb_weights = dense1_weights[:2048]
hha_weights = dense1_weights[-576:]

In [13]:
for name, weights in [('rgb', rgb_weights), ('hha', hha_weights)]:
    print(f"\n\n{'-'*30}\n{name}:")

    print(f'avg: {np.average(np.abs(weights))}')
    print(f'median: {np.median(np.abs(weights))}')
    print(f'max: {np.max(weights)}')
    print(f'min: {np.min(weights)}')
    print(f'std: {np.std(weights)}')



------------------------------
rgb:
avg: 0.023348242044448853
median: 0.023321080952882767
max: 0.0466926172375679
min: -0.0466923713684082
std: 0.026955856010317802


------------------------------
hha:
avg: 0.023346219211816788
median: 0.02336551435291767
max: 0.04668945074081421
min: -0.04669182375073433
std: 0.026949765160679817


results: all statistics almost identical...

### rgb-d

In [14]:
rgbd_mdl.summary()

Model: "rgbd_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
depth_input (InputLayer)        [(None, 200, 200, 1) 0                                            
__________________________________________________________________________________________________
normalize_depth (Rescaling)     (None, 200, 200, 1)  0           depth_input[0][0]                
__________________________________________________________________________________________________
depth_conv1 (Conv2D)            (None, 200, 200, 32) 320         normalize_depth[0][0]            
__________________________________________________________________________________________________
depth_maxpool1 (MaxPooling2D)   (None, 100, 100, 32) 0           depth_conv1[0][0]                
_________________________________________________________________________________________

In [15]:
dense1_weights = rgbd_mdl.get_layer(name='dense1').weights[0]
dense1_weights

<tf.Variable 'dense1/kernel:0' shape=(2624, 128) dtype=float32, numpy=
array([[ 0.01389671, -0.00398814,  0.00222813, ..., -0.01448179,
         0.14257964, -0.0127115 ],
       [-0.00167028, -0.12044226, -0.03236815, ..., -0.01964624,
         0.02263955,  0.0066269 ],
       [-0.10899132, -0.09754838, -0.02063757, ..., -0.04244673,
         0.00974357, -0.03693372],
       ...,
       [-0.02111984, -0.00108565,  0.02134815, ..., -0.03997415,
         0.03468368, -0.02181706],
       [ 0.03983618, -0.01885728, -0.04094834, ...,  0.01095223,
        -0.00877238, -0.01518761],
       [-0.00902191, -0.00205019,  0.0148508 , ...,  0.03920665,
         0.01647745, -0.04122907]], dtype=float32)>

In [16]:
rgb_weights = dense1_weights[:2048]
depth_weights = dense1_weights[-576:]

In [17]:
for name, weights in [('rgb', rgb_weights), ('depth', depth_weights)]:
    print(f"\n\n{'-'*30}\n{name}:")

    print(f'avg absolute: {np.average(np.abs(weights))}')
    print(f'median: {np.median(np.abs(weights))}')
    print(f'max: {np.max(weights)}')
    print(f'min: {np.min(weights)}')
    print(f'std: {np.std(weights)}')



------------------------------
rgb:
avg absolute: 0.04485052451491356
median: 0.029391683638095856
max: 0.8412381410598755
min: -0.9375953078269958
std: 0.0712394043803215


------------------------------
depth:
avg absolute: 0.024327704682946205
median: 0.02351420745253563
max: 0.16914620995521545
min: -0.16942475736141205
std: 0.028688514605164528


this time, statistics not identical. but they are similar (same order of magnitude). avg and median absolute values higher on rgb, and max/min significantly higher. std lower on depth.